# Tutorial 2: Tensor Decompositions
source https://www.tensors.net/j-tutorial-2

This tutorial covers the basics of decomposing tensors into products of other tensors, including:    

- Special tensor types: diagonal, unitary, isometric tensors

- Use of singular value decomposition 'svd' to decompose tensors

- Use of spectral decomposition 'eig' to decompose tensors

- Use of QR decomposition 'qr' to decompose tensors

-  Frobenius norm and optimal restricted rank tensor truncations

In [1]:
##### Julia
#
using LinearAlgebra
# ensure 'ncon.jl' in working dir
include("ncon.jl");

## T2.1: Special tensor types: diagonal, unitary, isometric 

We begin by considering some special types of tensor and the diagrammatic notation used to represent them:

<img  src="./Figs/Fig.2.1(a,b,c).png"  width="600"  align="center" />


Fig.2.1(a) shows the identity tensor **I**, equivalent to the Kronecker delta function δij, which is simply depicted as a line. Fig.2.1(b) shows a diagonal matrix **D** with elements {λ1, λ2, …}, which is depicted as a small solid circle (in contrast to non-diagonal matrices, which are represented as larger or colored shapes). Fig.2.1(c) shows a higher order generalization of a diagonal tensor, again depicted as a small solid circle, which takes value λi if the indices match, i = j = k, and is zero otherwise.

<img  src="./Figs/Fig.2.1(d,e).png"  width="600"  align="center" />

Fig.2.1(d) shows a unitary matrix **U**, which annihilates to the identity with its conjugate U† as depicted. We often draw the conjugate of a tensor with opposite vertical orientation (and here use arrows to explicitly denote tensor orientation for clarity). (e) We say that a tensor is unitary if there exists a bipartition of the indices (here denoted by the dotted line) under which the tensor could be reshaped into a unitary matrix. In many cases the shape of a tensor can be conveniently used to indicate its orientation, eliminating the need to use arrows for this purpose.

<img  src="./Figs/Fig.2.1(f,g).png"  width="600"  align="center" />

Fig.2.1(f) shows an isometric matrix **W**, where we have assumed that d1 > d2. When contracted on the larger dimensional index with its conjugate the identity is produced, **W†W = I**. However, contraction the other way, WW† = P, produces a projector **P**. A projector is a Hermitian tensor that squares to itself, as illustrated in Ex.2.1(g). These properties imply that the eigenvalues of a projector **P** must either be 0 or 1.

<img  src="./Figs/Fig.2.1(h,i).png"  width="600"  align="center" />


Fig.2.1(h-i) give examples of isometric tensors, which can be reshaped into isometric matrices under the partition indicated by the dotted lines, and the constraints that they satisfy.

**Diagrammatic conventions**: often a unitary tensor will be represented as a rectangle or trapezoid, where the longer dimension indicates the index partition under which it can be reshaped into a unitary matrix. Similarly, isometries are often represented as wedges which 'point' in the direction of the smaller dimension (i.e such that they annihilate to identity when contracted with their conjugate along the base of the wedge). These conventions allow one to indicate the partition of indices under which a tensor becomes isometric/unitary by its shape alone, eliminating the need to indicate these partitions explicitly. However, not all tensor network researchers follow these conventions.

## T2.2: Tensor decompositions


In this section we discuss how several common matrix decompositions, including the singular value decomposition (SVD), the spectral decomposition and the QR decomposition are generalized to the case of tensors. These decompositions play a key role in many tensor network algorithms, particularly for their use in schemes to approximate a high-order tensor as a product of low-order tensors. 



​We begin with the SVD, a widely used decomposition in many areas of mathematics, statistics, physics and engineering, which is defined for matrices of arbitrary dimension. Below are several examples:

<img  src="./Figs/Fig.2.2(a,b).png"  width="600"  align="center" />

Fig.2.2(a) depicts the singular value decomposition of a d1×d2 matrix **A** (where we have assumed that d1 > d2) into a product of matrices, **A = USV†**. Here **U** is a d1×d2 isometric matrix, **V** is a d2×d2 unitary matrix, and **S** is a d2×d2 diagonal matrix with positive, real elements ordered with descending magnitude (called the singular values). The cost of performing the SVD scales the square of the smaller matrix dimension times the larger dimension, i.e. as **O(d1d2^2)**. Here we are using the 'economical' or 'thin' form of SVD, otherwise the 'full' SVD would give both U and V as unitary matrices and S as a d1×d2 diagonal matrix that is padded with zeros on the larger dimension.

In [12]:
##### Ex2.2(a): SVD of matrix
d1 = 10; d2 = 6;
A = rand(d1,d2);
F = svd(A);
# check result
Af = F.U*diagm(0 => F.S)*F.Vt;
dA = norm(Af[:]-A[:])

Fig.2.2(b) depicts the SVD of a order-3 tensor **A** across the partition of indices specified by the dotted line. After using the appropriate 'reshape' commands the decomposition proceeds similarly to the previous example, except that tensor **U** is reshaped into a order-3 isometry.

In [13]:
##### Ex2.2(b): SVD of tensor
d = 10; A = rand(d,d,d);
F = svd(reshape(A,d^2,d));
U = reshape(F.U,d,d,d);
# check result
Af = ncon(Any[U,diagm(0 => F.S),F.Vt],
    Any[[-1,-2,1],[1,2],[2,-3]]);
dA = norm(Af[:]-A[:])

1.0436466959934603e-13

**Aside**:

the SVD is also useful for generating random unitary and isometric tensors.

In [3]:
##### Initialize unitaries and isometries
d1 = 10; d2 = 6;

# d1-by-d1 random unitary matrix U
U = svd(rand(d1,d1)).U

# d1-by-d2 random isometric matrix W
W = svd(rand(d1,d2)).U

10×6 Matrix{Float64}:
 -0.348262   -0.561862    0.0512832   0.113969     0.334726    0.263928
 -0.288453   -0.510327   -0.0644296  -0.341748    -0.165522    0.00160228
 -0.25519     0.266434   -0.108591    0.282248    -0.0945064   0.687302
 -0.390314   -0.0515584  -0.126104    0.0406522   -0.149296    0.147348
 -0.309037   -0.0824208   0.251883    0.415377     0.424614   -0.458351
 -0.285558    0.0345505  -0.650896    0.0922701   -0.326243   -0.447766
 -0.349285    0.435949   -0.305887   -0.00831072   0.430635    0.0131768
 -0.0997924  -0.124547   -0.147748   -0.377292    -0.037754    0.0878447
 -0.321951    0.0725206   0.45388     0.317834    -0.594362   -0.0844356
 -0.406674    0.362522    0.396245   -0.603667     0.0496983  -0.103171

The **spectral decomposition** (or eigen-decomposition) is a particularly useful decomposition for  Hermitian matrices or tensors where we desire that the decomposition explicitly reserves the Hermtian character. Below are several examples: 

<img  src="./Figs/Fig.2.2(c,d).png"  width="600"  align="center" />

In [4]:
##### Ex2.2(c): spect. decomp. of matrix
d = 10; A = rand(d,d);
H = 0.5*(A+A'); #random Hermitian
F = eigen(H);
# check result
Hf = F.vectors*diagm(0 => F.values)*F.vectors'
dH = norm(Hf[:]-H[:])

6.63923525338072e-15

In [ ]:
##### Ex2.2(d): spect. decomp. of tensor
d = 2; A = rand(d,d,d,d);
H = 0.5*(A + permutedims(A,[3,4,1,2]));
F = eigen(reshape(H,d^2,d^2));
U = reshape(F.vectors,d,d,d^2);
D = diagm(0 => F.values);
# check result
Hf = ncon(Any[U,D,U],
    Any[[-1,-2,1],[1,2],[-3,-4,2]]);
dH = norm(Hf[:]-H[:])

<img  src="./Figs/Fig.2.2(e).png"  width="600"  align="center" />

**Aside**: For any matrix **A** the spectral decompositions of **AA†** and **A†A** are related to the SVD of **A** as depicted in Fig.2.2(e). More precisely, the eigenvectors **U** of **AA†** and **V** of **A†A** (trimmed to remove contributions from trivial eigenvalues) are equal to the singular vectors contained in **U** and **V** from the SVD, and the (non-zero) eigenvalues in **D** are the squares of the singular values in **S**.  

The final decomposition we consider is the **QR decomposition** which is useful to orthogonalize a tensor network, as will be discussed later in Tutorial 3.

<img  src="./Figs/Fig.2.2(f).png"  width="600"  align="center" />


Fig.2.2(f) depicts the QR decomposition of a d1×d2 matrix A (where we have assumed that d1 > d2) into a product of matrices, A = QR. Here Q is a d1×d2 isometric matrix and R is a d2×d2 upper-triangular matrix. The cost of performing the QR scales as O(d1d2^2), which is the same as the SVD, although the QR decomposition tends to be faster in practice by a constant factor. Here we are using the so-called 'economical' QR decomposition, otherwise the 'full' QR would give both Q as a d1×d1 unitary matrix and would pad R with zeros. Fig.2.2(g) depicts the QR decomposition of a order-3 tensor A across the partition of indices specified by the dotted line. The decomposition proceeds similarly to the previous example, except that tensor Q is reshaped into a order-3 isometry.

In [5]:
##### Ex2.2(f): QR decomp of matrix
d1 = 10; d2 = 6;
A = rand(d1,d2);
F = qr(A);
# check result
Af = F.Q*F.R;
dA = norm(Af[:]-A[:])

3.06033541867706e-15

In [7]:
##### Ex2.2(g): QR decomp of tensor
d = 10;
A = rand(d,d,d);
F = qr(reshape(A,d^2,d));
Q = reshape(Array(F.Q),d,d,d);
# check result
Af = ncon(Any[Q,F.R],Any[[-1,-2,1],[1,-3]]);
dA = norm(Af[:]-A[:])

7.158070107318275e-15

## T2.3: Frobenius norm for tensors

The matrix decompositions in the previous section are useful for approximating a single tensor as a product multiple tensors. However, before discussing such approximations it is necessary to define the tensor norm that we use. A tensor norm that is particular useful for us is the Frobenius norm (or Hilbert-Schmidt norm). Given a matrix Aij the norm ‖A‖ is defined as the square-root of the sum of the magnitude of each element squared:

<img  src="./Figs/Fig.2.3(a,b,c).png"  width="600"  align="center" />

In [8]:
##### Ex2.3(c)
d = 10;
A = rand(10,10,10,10,10);
# frobenus norm
frobA0 = sqrt(ncon(Any[A,conj(A)],Any[collect(1:ndims(A)),collect(1:ndims(A))]))
# equivalent frobenus norm
frobA1 = sqrt(sum(abs.(A[:]).^2))
# also equivalent frobenus norm
frobA2 = norm(A)
(frobA0,frobA1,frobA2)

(182.69843276910777, 182.69843276910782, 182.6984327691068)

Fig.2.3(c) depicts the tensor trace Ttr of tensor A, which is the product of A and its conjugate A† where all matching indices are contracted. A useful property of the Frobenius norm is that it has a simple expression in terms of the singular values of a tensor. Let A be a tensor whose singular values across some chosen partition are {s1, s2, s2,...}, as depicted in Fig.2.3(d), then it follows that Ttr(AA†) is simply the square root of the sum of the singular values squared: 

<img  src="./Figs/Fig.2.3(d,e).png"  width="600"  align="center" />

Let us assume that we wish to approximate some tensor A with another tensor B of equal size. It is useful to define the error of approximation ε as the difference between the tensors in the Frobenius norm divided by the norm of A: 

<img  src="./Figs/Fig.2.3(d).png"  width="600"  align="center" />

## T2.4: Restricted rank tensor approximations

Given some tensor A and some bi-partition of its indices, we define the decomposition rank r as the minimal internal dimension such that A can be expressed as a product of tensors, A = B ⋅ C, across this partition  <font color=red> (i.e. rank r is the dimension of the index contracted between B and C) </font>. Although we will not prove this here, it can be shown that the decomposition rank r of tensor A across a partition precisely equals the number of non-zero singular values in the SVD across the partition. 

<img  src="./Figs/Fig.2.4(a,b).png"  width="600"  align="center" />

In [8]:
##### Ex2.4(a): take SVD
d = 10; A = rand(d,d,d,d,d);
F = svd(reshape(A,d^3,d^2));
U = reshape(F.U,d,d,d,d^2);
Vh = reshape(F.Vt,d^2,d,d);
##### truncate SVD
chi = 8;
Vhtilda = Vh[1:chi,:,:];
Stilda = diagm(0 => F.S[1:chi]);
Utilda = U[:,:,:,1:chi];
B = ncon(Any[Utilda,Stilda,Vhtilda],Any[[-1,-2,-3,1],[1,2],[2,-4,-5]]);
##### compare
epsAB = norm(A-B) / norm(A)

0.4685393101593581


Following Fig.2.4(a), the truncation error ε of the optimal restricted rank approximation is simply given as the square-root of the sum of the squares of the discarded singular values, as 
$$
\varepsilon=\sqrt{\sum_{k>\chi}(s_k)^2} 
$$
 (assuming that the starting tensor was properly normalized, ‖A‖ = 1).  

A useful concept is the effective rank to accuracy Δ of a tensor, denoted as r(Δ), which is defined as the number of singular values greater than or equal to Δ across the decomposition. In many cases (particularly if the spectrum of singular values is sharply decaying) then the error ε in truncating a tensor to rank r(Δ) will be ε ≈ Δ, since the expression Fig.2.4(c) will be dominated by the largest singular value that is truncated. Thus the value Δ often serves as a useful proxy for the true truncation error ε.

In [13]:
##### Ex2.4(d): effective rank
# Generate toeplitz matrix
d = 500;
A = diagm( 1 => ones(d-1)) +
    diagm(-1 => ones(d-1));
A = A / norm(A) #normalize

# compute effective rank to accuracy 'deltaval'
deltaval = 1e-2;
F = svd(A);
r_delta = sum(F.S .> deltaval)
eps_err = sqrt.(sum(F.S[r_delta+1:end] .^2))

0.040409915684386455

In Ex.2.4(d) we generate a Toeplitz matrix A and compute the effective rank at accuracy Δ = 1e-2, which evaluates to r(Δ = 1e-2) = 450. We then compute the truncation error of the restricted rank approximation, which is found to be ε ≈ 4e-2.

**Aside**: Notice also that if a tensor H was Hermitian and positive semi-definite w.r.t the index partition, then one could obtain the optimal restricted rank approximation via the spectral decomposition by truncating the smallest weight eigenvalues. This follows from the relationship between the eigenvalues and singular values of matrices as discussed in Fig.2.2(e). The factorization given from the spectral decomposition can be more useful than that from the SVD in this context, as the former explicitly preserves the Hermitian character of H, although in principle both decompositions would have the same accuracy ε.

## Outlook: Tensor Decompositions

Optimal restricted rank tensor approximations, obtained via the SVD or the spectral decomposition, play a central role in tensor network theory and are also essential ingredients to most tensor network algorithms. In fact, most common tensor network algorithms are simply composed of some combination of contraction steps, as discussed in Tutorial 1, together with some combination of tensor decompositions. Thus we have already learned the foundational tools of tensor network methods! Subsequent tutorials will discuss how these tools can be applied and composed into algorithms for certain tasks...

## Problem Set 2:

<img  src="./Figs/Pb.2.png"  width="600"  align="center" />

Pb.2: Tensor A is an order-4 tensor that we define element-wise as given above. Note that the difference between the MATLAB/Julia and Python definitions follows from the use of 1-based indexing in the former versus the use 0-based indexing in the latter, but it is still the same tensor between all three programming languages.

​

(a) Assume that indices i, j are of dimension d1 and indices k, l are of dimension d2 (with d2 < d1). How does the cost of taking the SVD across the indicated partition scale with d1 and d2?

​

(b) Generate the tensor A for d1 = 10 and d2 = 8. What is the norm ‖A‖? After computing the norm construct the normalized tensor: A' = A / ‖A‖.

​

(c) Take the SVD of A' across the indicated partition. Check that the square root of the sum of the singular values squared is equal to 1. Why is this the case?

​

(d) What is the effective rank r(Δ) of A' at Δ = 1e-4 ?

​

(e) Compute the truncation error ε of the restricted rank approximation r(Δ=1e-4) indirectly using the singular values as per Fig.2.4(c)

​

(f) Construct the optimal restricted rank approximation to A' via the truncated SVD. Compute the truncation error ε of this approximation and check that your answer is consistent with part (e).

<img  src="./Figs/Pb.2.ans.png"  width="600"  align="center" />

In [14]:
##### Pb.2(a): solutions
using LinearAlgebra
# ensure 'ncon.jl' in working dir
include("ncon.jl");

##### Define matrix A
d1 = 10;
d2 = 8;
A = zeros(d1,d1,d2,d2);
for ni = 1:d1
    for nj = 1:d1
        for nk = 1:d2
            for nl = 1:d2
                A[ni,nj,nk,nl] = sqrt(ni + 2*nj + 3*nk + 4*nl);
            end
        end
    end
end

##### Compute norm and normalize A
n_A = sqrt.(sum(A[:] .^2))
Ap = A./n_A;

##### take SVD of A
F = svd(reshape(Ap,d1^2,d2^2));
m_A = sqrt.(sum(F.S .^2)) #should equal 1

##### effective rank
deltaval = 1e-4;
r_delta = sum(F.S .> deltaval)

##### truncation error
err_0 = sqrt.(sum((F.S[r_delta+1:end]) .^2));

##### optimal restricted rank approximation
B = F.U[:,1:r_delta]*diagm(0 => F.S[1:r_delta])*F.Vt[1:r_delta,:];
B = reshape(B,d1,d1,d2,d2);
err_1 = sqrt.(sum((Ap[:] - B[:]) .^2));

# compare
[err_0,err_1]

2-element Vector{Float64}:
 1.0484615133220966e-5
 1.0484615133211092e-5